In [ ]:
import tensorflow as tf 
import numpy as np 
import os 
import pandas as pd 
import re
import string
import nltk
import matplotlib.pylab as plt 
tf.config.run_functions_eagerly(run_eagerly=True)


In [ ]:
train_dir = "/kaggle/input/imdbcomplete/aclImdb/train"
val_dir = "/kaggle/input/imdbcomplete/aclImdb/test"

In [ ]:
## Preprocess input strings:
 ## Remove html tags
 ## remove punctuation, spaces
 ## Normalize words(Lemmatizer) Better=> Good
def process_string(input):
    output = input.lower()
    output = re.sub("<[^>]+>",'',output)
    output = re.sub("[%s]"%re.escape(string.punctuation),' ',output)
    output = re.sub("  ",' ',output)
    lemmatizer = nltk.stem.WordNetLemmatizer()
    output = ' '.join([lemmatizer.lemmatize(w,pos="a") for w in output.split()])
    return output 

def pr_sent(input):
    output = tf.strings.lower(input)
    output =tf.strings.regex_replace(output,"<[^>]+>","")
    output =tf.strings.regex_replace(output,"[%s]"%re.escape(string.punctuation),' ')
    output =tf.strings.regex_replace(output,"  "," ")
    return output

In [ ]:
from tensorflow.keras.preprocessing import text_dataset_from_directory
train_dataset_ = text_dataset_from_directory(train_dir, batch_size =8, shuffle=True, )
val_dataset_ = text_dataset_from_directory(val_dir, batch_size =8, shuffle=True)

In [ ]:
# Applying the preprcessing methods
def pr(r,w):
    return pr_sent(r),w
train_dataset = train_dataset_.map(pr)
val_dataset = val_dataset_.map(pr)
train_dataset = train_dataset.prefetch(buffer_size=32).cache().shuffle(buffer_size=25000)
val_dataset = val_dataset.prefetch(buffer_size=32).cache().shuffle(buffer_size=25000)


In [ ]:
# TextVectorization will transform words into onehot encoding style
# with a vocabulary of 10 000 words and a sequence length of 100 w
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
vec_layer = TextVectorization(
    max_tokens=10000,
    output_mode ='int',
    output_sequence_length =100
)
td = train_dataset.map(lambda x,y:x)
vec_layer.adapt(td)

In [ ]:
def vectorizer(review,label):
    return tf.one_hot(vec_layer(review),depth=10000),label
train_dataset = train_dataset.map(vectorizer)
val_dataset = val_dataset.map(vectorizer)

In [ ]:
# Checking the preprocessing phase
for r,w in train_dataset.take(1):
    print(r,w)
print(train_dataset)